In [1]:
## Introduction & Model Setup
from langchain_aws.chat_models import ChatBedrockConverse

model = ChatBedrockConverse(model="us.meta.llama3-1-70b-instruct-v1:0", region_name="us-east-1")
model

ChatBedrockConverse(disable_streaming='tool_calling', client=<botocore.client.BedrockRuntime object at 0x77d7ca364430>, model_id='us.meta.llama3-1-70b-instruct-v1:0', region_name='us-east-1', provider='us', supports_tool_choice_values=())

In [2]:
## Few Shot Prompting
from langchain_core.prompts import (ChatPromptTemplate,
                                    FewShotChatMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    SystemMessagePromptTemplate)

sys = SystemMessagePromptTemplate.from_template("You are an expert instructional agent who replies briefly.")

examples = [
    {"human_message": "How would I get the number of items in a Python list?", "ai_message": "len"},
    {"human_message": "How do I multiply two numbers in Python?", "ai_message": "*"},
    {"human_message": "What is the Python keyword for defining a function?", "ai_message": "def"},
]
example_prompt = ChatPromptTemplate.from_messages([("human", "{human_message}"), ("ai", "{ai_message}")])
examples_prompt = FewShotChatMessagePromptTemplate(examples=examples, example_prompt=example_prompt)

question = HumanMessagePromptTemplate.from_template("How do I determine what class an object is in python?")

full_prompt = sys + examples_prompt + question
display(full_prompt.format_messages())

few_shot_chain = full_prompt | model
no_examples_chain = (sys + question) | model

few_shot_results = few_shot_chain.invoke({})
no_examples_results = no_examples_chain.invoke({})

print("Few Shot Prompting Response:", few_shot_results.content)
print("\nNo Examples Prompting Response:", no_examples_results.content)

[SystemMessage(content='You are an expert instructional agent who replies briefly.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How would I get the number of items in a Python list?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='len', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How do I multiply two numbers in Python?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='*', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the Python keyword for defining a function?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='def', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How do I determine what class an object is in python?', additional_kwargs={}, response_metadata={})]

Few Shot Prompting Response: 

type()

No Examples Prompting Response: 

You can use the `type()` function to determine the class of an object in Python. For example:

`obj = "Hello, World!"`
`print(type(obj))  # Output: <class 'str'>`

Alternatively, you can use the `isinstance()` function to check if an object is an instance of a specific class:

`print(isinstance(obj, str))  # Output: True`


In [3]:
## Chain of Thought Prompting
task = "What is the fourth most populous state in the united states? {addition_instructions}"
prompt = ChatPromptTemplate([("human", task)])
chain = prompt | model

results = chain.batch(
    [
        {"addition_instructions": ""},
        {"addition_instructions": "Let's think step by step."},
    ]
)

print("*Without COT*")
print(results[0].content)
print("\n*With COT*")
print(results[1].content)

*Without COT*


According to the United States Census Bureau (2020 estimates), the fourth most populous state in the United States is Florida, with an estimated population of approximately 21.76 million people.

Here are the top 5 most populous states in the US:

1. California - approximately 39.51 million people
2. Texas - approximately 29.74 million people
3. Florida - approximately 21.76 million people
4. New York - approximately 20.20 million people
5. Pennsylvania - approximately 12.77 million people

Note: These numbers are estimates based on 2020 data and may have changed slightly since then due to various factors such as population growth, migration, and other demographic changes.

*With COT*


To determine the fourth most populous state in the United States, let's break it down step by step:

**Step 1: Identify the 50 states and their populations**
According to the United States Census Bureau, the 50 states and their estimated populations as of 2020 are:

**Step 2: Sort the st